In [1]:
!pip install tensorflow

In [35]:
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN,Dense,Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [59]:
import pandas as pd
sentences=pd.read_csv("/content/earth.txt",sep=".")
sentences

,"Earth has a dynamic atmosphere, which sustains Earth's surface conditions and protects it from most meteoroids and UV-light at entry",It has a composition of primarily nitrogen and oxygen,"Water vapor is widely present in the atmosphere, forming clouds that cover most of the planet","The water vapor acts as a greenhouse gas and, together with other greenhouse gases in the atmosphere, particularly carbon dioxide (CO2), creates the conditions for both liquid surface water and water vapor to persist via the capturing of energy from the Sun's light",


In [60]:
#Tokeninzing the words
tokenizer=Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1
print(total_words)

60


In [61]:
# Creating input sequences and their corresponding next words
input_sequences = []
for sentence in sentences:
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(tokenized_sentence)):
        n_gram_sequence = tokenized_sentence[:i+1]
        input_sequences.append(n_gram_sequence)
input_sequences

[[17, 8],
 [17, 8, 4],
 [17, 8, 4, 18],
 [17, 8, 4, 18, 5],
 [17, 8, 4, 18, 5, 19],
 [17, 8, 4, 18, 5, 19, 20],
 [17, 8, 4, 18, 5, 19, 20, 21],
 [17, 8, 4, 18, 5, 19, 20, 21, 9],
 [17, 8, 4, 18, 5, 19, 20, 21, 9, 10],
 [17, 8, 4, 18, 5, 19, 20, 21, 9, 10, 2],
 [17, 8, 4, 18, 5, 19, 20, 21, 9, 10, 2, 22],
 [17, 8, 4, 18, 5, 19, 20, 21, 9, 10, 2, 22, 11],
 [17, 8, 4, 18, 5, 19, 20, 21, 9, 10, 2, 22, 11, 12],
 [17, 8, 4, 18, 5, 19, 20, 21, 9, 10, 2, 22, 11, 12, 13],
 [17, 8, 4, 18, 5, 19, 20, 21, 9, 10, 2, 22, 11, 12, 13, 23],
 [17, 8, 4, 18, 5, 19, 20, 21, 9, 10, 2, 22, 11, 12, 13, 23, 2],
 [17, 8, 4, 18, 5, 19, 20, 21, 9, 10, 2, 22, 11, 12, 13, 23, 2, 24],
 [17, 8, 4, 18, 5, 19, 20, 21, 9, 10, 2, 22, 11, 12, 13, 23, 2, 24, 14],
 [17, 8, 4, 18, 5, 19, 20, 21, 9, 10, 2, 22, 11, 12, 13, 23, 2, 24, 14, 25],
 [17,
  8,
  4,
  18,
  5,
  19,
  20,
  21,
  9,
  10,
  2,
  22,
  11,
  12,
  13,
  23,
  2,
  24,
  14,
  25,
  26],
 [11, 8],
 [11, 8, 4],
 [11, 8, 4, 27],
 [11, 8, 4, 27, 6],
 [11,

In [62]:
# Padding sequences for consistent input size
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')
input_sequences

array([[ 0,  0,  0, ...,  0, 17,  8],
       [ 0,  0,  0, ..., 17,  8,  4],
       [ 0,  0,  0, ...,  8,  4, 18],
       ...,
       [ 0,  0,  1, ..., 58, 12,  1],
       [ 0,  1,  3, ..., 12,  1, 59],
       [ 1,  3,  7, ...,  1, 59, 14]], dtype=int32)

In [63]:
# Creating input and output data
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)

In [64]:
# Building a simple RNN model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=50, input_length=max_sequence_length-1))
model.add(SimpleRNN(100, return_sequences=True))
model.add(SimpleRNN(100))
model.add(Dense(total_words, activation='softmax'))


In [65]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X,y,epochs=5,verbose=2)

Epoch 1/5
3/3 - 2s - loss: 4.2019 - accuracy: 0.0235 - 2s/epoch - 727ms/step
Epoch 2/5
3/3 - 0s - loss: 3.9341 - accuracy: 0.1529 - 75ms/epoch - 25ms/step
Epoch 3/5
3/3 - 0s - loss: 3.7825 - accuracy: 0.1294 - 105ms/epoch - 35ms/step
Epoch 4/5
3/3 - 0s - loss: 3.6440 - accuracy: 0.1647 - 140ms/epoch - 47ms/step
Epoch 5/5
3/3 - 0s - loss: 3.5187 - accuracy: 0.2941 - 125ms/epoch - 42ms/step


In [66]:

# Generating text using the trained model
seed_text = input("Enter the starting word: ")
next_words = int(input("Enter how many words to predict: "))

for _ in range(next_words):
    tokenized_seed = tokenizer.texts_to_sequences([seed_text])[0]
    tokenized_seed = pad_sequences([tokenized_seed], maxlen=max_sequence_length-1, padding='pre')
    predicted_word_index = np.argmax(model.predict(tokenized_seed), axis=-1)
    predicted_word = tokenizer.index_word[predicted_word_index[0]]
    seed_text += " " + predicted_word

print(seed_text)

Enter the starting word: almost
Enter how many words to predict: 12
1/1 [==============================] - 0s 29ms/step
almost a a a a the the the the the the the the


In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [15]:
#define image size and batch size
IMG_SIZE=224
BATCH_SIZE=32

In [16]:
#creating training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [26]:
train_generator = train_datagen.flow_from_directory(
    r'/content/drive/MyDrive/Brain_Tumor_Detection/Train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)


Found 246 images belonging to 2 classes.


In [18]:
#creating validation data with above paramenter
#folder=parameter.flow_from_directory(path,ts,bs.cm,subset)
val_generator=train_datagen.flow_from_directory(
   r'/content/drive/MyDrive/Brain_Tumor_Detection/Train',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

Found 61 images belonging to 2 classes.


In [10]:
model=keras.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(1,activation='sigmoid')

])

In [11]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [20]:
model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
8/8 [==============================] - 70s 9s/step - loss: 1.6655 - accuracy: 0.4797 - val_loss: 0.6382 - val_accuracy: 0.7377
Epoch 2/5
8/8 [==============================] - 28s 3s/step - loss: 0.5994 - accuracy: 0.6911 - val_loss: 0.4715 - val_accuracy: 0.7869
Epoch 3/5
8/8 [==============================] - 27s 3s/step - loss: 0.5121 - accuracy: 0.7317 - val_loss: 0.4232 - val_accuracy: 0.7869
Epoch 4/5
8/8 [==============================] - 27s 3s/step - loss: 0.6270 - accuracy: 0.7114 - val_loss: 0.4359 - val_accuracy: 0.8197
Epoch 5/5
8/8 [==============================] - 28s 3s/step - loss: 0.4649 - accuracy: 0.7764 - val_loss: 0.4069 - val_accuracy: 0.7869


In [22]:
model.save('model.h5','label.txt')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# New Section